# Analyze A/B Test Results 

This project will assure you have mastered the subjects covered in the statistics lessons. We have organized the current notebook into the following sections: 

- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)

<a id='intro'></a>
## Introduction

A/B tests are very commonly performed by data analysts and data scientists. For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should:
- Implement the new webpage, 
- Keep the old webpage, or 
- Perhaps run the experiment longer to make their decision.

<a id='probability'></a>
## Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

random.seed(42)

In [14]:
def print_percentage(text, decimal):
    print(f'{text}: {round(decimal * 100, 2)}%')

Below is the description of the data, there are a total of 5 columns:

<center>

|Data columns|Purpose|Valid values|
| ------------- |:-------------| -----:|
|user_id|Unique ID|Int64 values|
|timestamp|Time stamp when the user visited the webpage|-|
|group|In the current A/B experiment, the users are categorized into two broad groups. <br>The `control` group users are expected to be served with `old_page`; and `treatment` group users are matched with the `new_page`. <br>However, **some inaccurate rows** are present in the initial data, such as a `control` group user is matched with a `new_page`. |`['control', 'treatment']`|
|landing_page|It denotes whether the user visited the old or new webpage.|`['old_page', 'new_page']`|
|converted|It denotes whether the user decided to pay for the company's product. Here, `1` means yes, the user bought the product.|`[0, 1]`|
</center>

**a.** First 5 rows of the dataset

In [2]:
df = pd.read_csv('ab_data.csv')
df.head()

user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1

**b.** The number of rows in the dataset.

In [3]:
df.shape[0]

294478

**c.** The number of unique users in the dataset.

In [4]:
df.user_id.drop_duplicates().count()

290584

**d.** The proportion of users converted.

In [51]:
print_percentage('Convertion rate', df.converted.mean())

Convertion rate: 11.97%


**e.** The number of times when the "group" is `treatment` but "landing_page" is not a `new_page`.

In [6]:
first_clause = 'group == "treatment" and landing_page != "new_page"'
second_clause = 'landing_page == "new_page" and group != "treatment"'
df.query(first_clause + ' or ' + second_clause).shape[0]

3893

**f.** Do any of the rows have missing values?

In [7]:
df.info() # There are no missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In a particular row, the **group** and **landing_page** columns should have either of the following acceptable values:

|user_id| timestamp|group|landing_page|converted|
|---|---|---|---|---|
|XXXX|XXXX|`control`| `old_page`|X |
|XXXX|XXXX|`treatment`|`new_page`|X |


It means, the `control` group users should match with `old_page`; and `treatment` group users should matched with the `new_page`. 

However, for the rows where `treatment` does not match with `new_page` or `control` does not match with `old_page`, we cannot be sure if such rows truly received the new or old page.  

In [8]:
# Remove the inaccurate rows, and store the result in a new dataframe df2

# First getting the indexes for wrong landing_page
indexes_wrong_page = df.query(first_clause).index.tolist()

# Second getting the indexes for wrong group
indexes_wrong_group = df.query(second_clause).index.tolist()

# Joining both lists to get all indexes with mismatched group and landing_page
indexes_to_drop = indexes_wrong_page + indexes_wrong_group
df2 = df.drop(index=indexes_to_drop)

df2.head()

user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1

In [9]:
# Double Check all of the incorrect rows were removed from df2 - 
# Output of the statement below should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

**a.** How many unique **user_id**s are in **df2**?

In [10]:
df2.user_id.drop_duplicates().count()

290584

**b.** There is one **user_id** repeated in **df2**.  What is it?

In [11]:
# Grouping by user_id can show which one is duplicated
df2.groupby('user_id').count().query('group != 1') # duplicated id: 773192

timestamp  group  landing_page  converted
user_id                                           
773192           2      2             2          2

**c.** Display the rows for the duplicate **user_id**? 

In [12]:
df2.query('user_id == 773192')

user_id                   timestamp      group landing_page  converted
1899   773192  2017-01-09 05:37:58.781806  treatment     new_page          0
2893   773192  2017-01-14 02:55:59.590927  treatment     new_page          0

**d.** Remove **one** of the rows with a duplicate **user_id**, from the **df2** dataframe.

In [13]:
# Remove one of the rows with a duplicate user_id
# In this case I chose to keep the newest
df2 = df2.drop(index=1899)

# Check again if the row with a duplicate user_id is deleted or not
df2.query('user_id == 773192')

user_id                   timestamp      group landing_page  converted
2893   773192  2017-01-14 02:55:59.590927  treatment     new_page          0

**a.** What is the probability of an individual converting regardless of the page they receive?<br><br>

In [15]:
p_population = df2.converted.mean()
print_percentage('General conversion', p_population)

General conversion: 11.96%


**b.** Given that an individual was in the `control` group, what is the probability they converted?

In [16]:
p_control = df2.query('group == "control"').converted.mean()
print_percentage('Control group conversion', p_control)

Control group conversion: 12.04%


**c.** Given that an individual was in the `treatment` group, what is the probability they converted?

In [17]:
p_treatment = df2.query('group == "treatment"').converted.mean()
print_percentage('Treatment group conversion', p_treatment)

Treatment group conversion: 11.88%


In [18]:
# Calculate the actual difference (obs_diff) between the conversion rates for the two groups.
obs_diff = p_treatment - p_control
print_percentage('Conversion difference between groups', obs_diff)

Conversion difference between groups: -0.16%


> A *negative* difference means the **control** group has a *higher conversion rate* than **treatment** group.

**d.** What is the probability that an individual received the new page?

In [19]:
total_new_page = df2.query('landing_page == "new_page"').shape[0]
total_population = df2.shape[0]

print_percentage('Probability for getting new_page', total_new_page / total_population)

Probability for getting new_page: 50.01%


**e.** Consider your results from parts (a) through (d) above, and explain below whether the new `treatment` group users lead to more conversions.

> From the statistics gotten, we can see that with almost the same proportions in the population (half is in the *control* group and half is in the *treatment* group), there is a very low observed difference in the conversion rate, pointing that the **old site** is 0.16% better than the **new site**.

<a id='ab_test'></a>
## Part II - A/B Test

Since a timestamp is associated with each event, you could run a hypothesis test continuously as long as you observe the events. 

However, then the hard questions would be: 
- Do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  
- How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  

For now, consider you need to make the decision just based on all the data provided.  

> Recall that you just calculated that the "converted" probability (or rate) for the old page is *slightly* higher than that of the new page (ToDo 1.4.c). 

If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should be your null and alternative hypotheses (**$H_0$** and **$H_1$**)?  

You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the "converted" probability (or rate) for the old and new pages respectively.

$$ H_{0}: p_{old} \geq p_{new} $$
$$ H_{1}: p_{old} < p_{new} $$

### Null Hypothesis $H_0$ Testing
Under the null hypothesis $H_0$, assume that $p_{new}$ and $p_{old}$ are equal. Furthermore, assume that $p_{new}$ and $p_{old}$ both are equal to the **converted** success rate in the `df2` data regardless of the page. So, our assumption is: <br><br>
<center>
$p_{new}$ = $p_{old}$ = $p_{population}$
</center>

> New hypotesis:

$$ H_{0}: p_{old} = p_{new} $$
$$ H_{1}: p_{old} \neq p_{new} $$

In [20]:
p_new = p_old = p_population # assumption

**a.** What is the **conversion rate** for $p_{new}$ under the null hypothesis? 

In [21]:
print_percentage('p_new conversion', p_new)

p_new conversion: 11.96%


**b.** What is the **conversion rate** for $p_{old}$ under the null hypothesis? 

In [22]:
print_percentage('p_old conversion', p_old)

p_old conversion: 11.96%


**c.** What is $n_{new}$, the number of individuals in the treatment group? <br><br>

In [23]:
n_new = df2.query('group == "treatment"').user_id.count()
n_new

145310

**d.** What is $n_{old}$, the number of individuals in the control group?

In [24]:
n_old = df2.query('group == "control"').user_id.count()
n_old

145274

**e. Simulate Sample for the `treatment` Group**<br> 
Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null hypothesis.  <br><br>

In [25]:
# Simulate a Sample for the treatment Group

# The only possible values for conversion is 1 and 0
a = [0, 1]

# The size of the simulation is the same as n_new
size = n_new

# The probability of converting is p_new, thus the probability
# of not converting is the inverse: 1 - p_new
p = [1 - p_new, p_new]

new_page_converted = np.random.choice(a, size=size, p=p)
print_percentage('Sample conversion for new_page', new_page_converted.mean())

Sample conversion for new_page: 11.89%


**f. Simulate Sample for the `control` Group** <br>
Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null hypothesis. <br> Store these $n_{old}$ 1's and 0's in the `old_page_converted` numpy array.

In [26]:
# Simulate a Sample for the control Group

# The control group is exactly the same as treatment group, just changing
# the variables, but not the possible outcomes

size = n_old
p = [1 - p_old, p_old]

old_page_converted = np.random.choice(a, size=size, p=p)
print_percentage('Sample conversion for old_page', old_page_converted.mean())

Sample conversion for old_page: 11.8%


**g.** Find the difference in the "converted" probability $(p{'}_{new}$ - $p{'}_{old})$ for your simulated samples from the parts (e) and (f) above. 

In [27]:
# Getting the mean of the simulated distribution gives us the probability,
# as the list is composed by 1 and 0 - 1 is a conversion and 0 a non-conversion.

# It will sum all conversions (1) and then divide by all elements, what is exactly
# the mean of the list.

simulated_diff = new_page_converted.mean() - old_page_converted.mean()
print_percentage('Simulated difference', simulated_diff)

Simulated difference: 0.09%



**h. Sampling distribution** <br>
Re-create `new_page_converted` and `old_page_converted` and find the $(p{'}_{new}$ - $p{'}_{old})$ value 10,000 times using the same simulation process you used in parts (a) through (g) above. 

<br>
Store all  $(p{'}_{new}$ - $p{'}_{old})$  values in a NumPy array called `p_diffs`.

In [28]:
# Sampling distribution 
p_diffs = []
a = [0, 1]

for _ in range(10000):
    # Getting the sample for different groups
    new_page_converted = np.random.choice(a, size=n_new, p=[1-p_new, p_new])
    old_page_converted = np.random.choice(a, size=n_old, p=[1-p_old, p_old])
    
    # Calculating the convertion rate for each group
    sim_p_new = new_page_converted.mean()
    sim_p_old = old_page_converted.mean()
    
    # Storing the difference in a list
    p_diffs.append(sim_p_new - sim_p_old)

p_diffs = np.array(p_diffs)
p_diffs

array([-0.00055248, -0.00265884,  0.00048651, ..., -0.0030926 ,
       -0.0016263 , -0.00011244])

**i. Histogram**<br> 
Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.<br><br>

Also, use `plt.axvline()` method to mark the actual difference observed  in the `df2` data (recall `obs_diff`), in the chart.  

In [29]:
plt.hist(p_diffs)
plt.axvline(obs_diff, color='r')
plt.title('Sampling Distribution of Differences in Conversion Rate (New vs. Old page)')
plt.xlabel('Difference of Conversion Rate (New vs. Old page)')
plt.ylabel('Number of Observations');

**j.** What proportion of the **p_diffs** are greater than the actual difference observed in the `df2` data?

In [30]:
(p_diffs > obs_diff).mean()

0.9097

**k.** Please explain in words what you have just computed in part **j** above.  
 - What is this value called in scientific studies?  
 - What does this value signify in terms of whether or not there is a difference between the new and old pages? *Hint*: Compare the value above with the "Type I error rate (0.05)". 

> The value calculated in part **j** is the ***p-value*** of the hypotesis test. Its purpose is to test what is the probability to get an extreme value in favor of the alternative hypotesis ($H_{1}$). We can use the *p-value* and compare it to our *Error Type 1 threshold ($\alpha$)* to make a decision about our hypotesis testing, as the following:

If ($ p_{value} \leq \alpha $): reject $H_{0}$

If ($ p_{value} > \alpha $): fail to reject $H_{0}$

> With a $p_{value} = 0.9097$, it is higher enough from our $\alpha = 0.05$ to fail to reject our null hypotesis ($p_{value} > \alpha$), which means there is no statistic evidence to say that $p_{new} \neq p_{old}$, or in other words, that there is difference in the **conversion rate** between the pages.



**l. Using Built-in Methods for Hypothesis Testing**<br>
We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. 

Fill in the statements below to calculate the:
- `convert_old`: number of conversions with the old_page
- `convert_new`: number of conversions with the new_page
- `n_old`: number of individuals who were shown the old_page
- `n_new`: number of individuals who were shown the new_page


In [31]:
import statsmodels.api as sm

# number of conversions with the old_page
convert_old = df2.query('landing_page == "old_page"').converted.sum()

# number of conversions with the new_page
convert_new = df2.query('landing_page == "new_page"').converted.sum()

# number of individuals who were shown the old_page
n_old = df2.query('landing_page == "old_page"').user_id.count()

# number of individuals who received new_page
n_new = df2.query('landing_page == "new_page"').user_id.count()

**m.** Now use `sm.stats.proportions_ztest()` to compute your test statistic and p-value.  [Here](https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html) is a helpful link on using the built in.

The syntax is: 
```bash
proportions_ztest(count_array, nobs_array, alternative='larger')
```
where, 
- `count_array` = represents the number of "converted" for each group
- `nobs_array` = represents the total number of observations (rows) in each group
- `alternative` = choose one of the values from `[‘two-sided’, ‘smaller’, ‘larger’]` depending upon two-tailed, left-tailed, or right-tailed respectively. 
>**Hint**: <br>
It's a two-tailed if you defined $H_1$ as $(p_{new} = p_{old})$. <br>
It's a left-tailed if you defined $H_1$ as $(p_{new} < p_{old})$. <br>
It's a right-tailed if you defined $H_1$ as $(p_{new} > p_{old})$. 

The built-in function above will return the z_score, p_value. 

---
### About the two-sample z-test
Recall that you have plotted a distribution `p_diffs` representing the
difference in the "converted" probability  $(p{'}_{new}-p{'}_{old})$  for your two simulated samples 10,000 times. 

Another way for comparing the mean of two independent and normal distribution is a **two-sample z-test**. You can perform the Z-test to calculate the Z_score, as shown in the equation below:

$$
Z_{score} = \frac{ (p{'}_{new}-p{'}_{old}) - (p_{new}  -  p_{old})}{ \sqrt{ \frac{\sigma^{2}_{new} }{n_{new}} + \frac{\sigma^{2}_{old} }{n_{old}}  } }
$$

where,
- $p{'}$ is the "converted" success rate in the sample
- $p_{new}$ and $p_{old}$ are the "converted" success rate for the two groups in the population. 
- $\sigma_{new}$ and $\sigma_{new}$ are the standard deviation for the two groups in the population. 
- $n_{new}$ and $n_{old}$ represent the size of the two groups or samples (it's same in our case)


>Z-test is performed when the sample size is large, and the population variance is known. The z-score represents the distance between the two "converted" success rates in terms of the standard error. 

Next step is to make a decision to reject or fail to reject the null hypothesis based on comparing these two values: 
- $Z_{score}$
- $Z_{\alpha}$ or $Z_{0.05}$, also known as critical value at 95% confidence interval.  $Z_{0.05}$ is 1.645 for one-tailed tests,  and 1.960 for two-tailed test. You can determine the $Z_{\alpha}$ from the z-table manually. 

Decide if your hypothesis is either a two-tailed, left-tailed, or right-tailed test. Accordingly, reject OR fail to reject the  null based on the comparison between $Z_{score}$ and $Z_{\alpha}$. 
>Hint:<br>
For a right-tailed test, reject null if $Z_{score}$ > $Z_{\alpha}$. <br>
For a left-tailed test, reject null if $Z_{score}$ < $Z_{\alpha}$. 


In other words, we determine whether or not the $Z_{score}$ lies in the "rejection region" in the distribution. A "rejection region" is an interval where the null hypothesis is rejected if the $Z_{score}$ lies in that region.



Reference: 
- Example 9.1.2 on this [page](https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/09%3A_Two-Sample_Problems/9.01%3A_Comparison_of_Two_Population_Means-_Large_Independent_Samples), courtesy www.stats.libretexts.org

In [32]:
import statsmodels.api as sm

# the number of converted for each group
counts_array = np.array([convert_new, convert_old])

# total number of observations (rows) in each group
nobs_array = np.array([n_new, n_old])

z_score, p_value = sm.stats.proportions_ztest(counts_array, nobs_array, alternative='larger')
print(z_score, p_value)

-1.3109241984234394 0.9050583127590245


**n.** What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

> Based on the values computed, we have the following findings:

* $z_{score} = -1.31092$
* $p_{value} = 0.90505$

> Setting a confidence interval of 95%, we get the critical value $Z_{0.05} = 1.645$, as we are assuming that is a right-tailed test ($H_1: p_{new} > p_{old}$). Therefore, using the decision making process with the $z_{score}$, we have the following: $Z_{score} < Z_{0.05}$. As the theory says, for a right-tailed test we should reject the *null hypotesis* if $Z_{score} > Z_{\alpha}$. We computed the oposite, thus we fail to reject $H_{0}$, impling that we do not have statistical evidence to say that the new page is better than the old page.

> Now, using the $p_{value} = 0.91$, we can make the same test as made earlier in letter **j**: taking the same 95% of confidence, we have an $\alpha = 0.05$. Then, this makes $p_{value} > \alpha$, what makes us fail to reject the *null hypotesis* again, reaching the same result as with the manual test.

<a id='regression'></a>
### Part III - A regression approach

In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

**a.** Since each row in the `df2` data is either a conversion or no conversion, what type of regression should you be performing in this case?

> We should use the **Logistic Regression**, as we are talking about the *conversion rate* of a given page. This means that we are not interested on predict a numeric value like amount of time spent on the page, but we want to find out the probability of a given user pay (convert) or not using that page - therefore, we are talking about only two possible values: positive (1) and negative (0).

**b.** The goal is to use **statsmodels** library to fit the regression model you specified in part **a.** above to see if there is a significant difference in conversion based on the page-type a customer receives. However, you first need to create the following two columns in the `df2` dataframe:
 1. `intercept` - It should be `1` in the entire column. 
 2. `ab_page` - It's a dummy variable column, having a value `1` when an individual receives the **treatment**, otherwise `0`.  

In [33]:
# Creating the intercept column to use in the model
df2['intercept'] = 1

# get_dummies returns two columns, but for making sure the
# we will get a reliable solution, we need to use only one of them
df2[['to_drop', 'ab_page']] = pd.get_dummies(df2['group'])
df2 = df2.drop('to_drop', axis=1)

df2.head()

user_id                   timestamp      group landing_page  converted  \
0   851104  2017-01-21 22:11:48.556739    control     old_page          0   
1   804228  2017-01-12 08:01:45.159739    control     old_page          0   
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0   
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0   
4   864975  2017-01-21 01:52:26.210827    control     old_page          1   

   intercept  ab_page  
0          1        0  
1          1        0  
2          1        1  
3          1        1  
4          1        0

**c.** Use **statsmodels** to instantiate your regression model on the two columns you created in part (b). above, then fit the model to predict whether or not an individual converts. 


In [34]:
lm = sm.Logit(df2['converted'], df2[['intercept', 'ab_page']])
results = lm.fit()

Optimization terminated successfully.
         Current function value: 0.366118
         Iterations 6


**d.** Provide the summary of your model below, and use it as necessary to answer the following questions.

In [35]:
results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            Pseudo R-squared: 0.000      
Dependent Variable: converted        AIC:              212780.3502
Date:               2021-11-24 22:33 BIC:              212801.5095
No. Observations:   290584           Log-Likelihood:   -1.0639e+05
Df Model:           1                LL-Null:          -1.0639e+05
Df Residuals:       290582           LLR p-value:      0.18988    
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     6.0000                                        
-------------------------------------------------------------------
              Coef.   Std.Err.      z      P>|z|    [0.025   0.975]
-------------------------------------------------------------------
intercept    -1.9888    0.0081  -246.6690  0.0000  -2.0046  -1.9730
ab_page      -0.0150    0.0114    -1.3109  0.1899  -0.0374   0.0074
==================================================================

"""

In [36]:
# When dealing with Logistic Regression, we need to
# exponentiate the coefficient to get the true meaning
np.exp(-0.0150)

0.9851119396030626

In [37]:
# As the exponentiated coefficient is less than 1, we can
# get the reciprocal to make it easier for the interpretation
1/np.exp(-0.0150)

1.015113064615719

> An user with the *old_page* has a 1.02% higher chance of convert than *new_page*

**e.** What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  

> The $p_{value}$ associated with *ab_page* is **0.1899**, and comparing it to the Type I Error of 95% ($\alpha = 0.05$), we find out that $p_{value} > \alpha$, what means that we do not have enough statistical evidence to reject $H_0$, exactly like on Part II. Thus, we cannot say that there is a significant difference of conversion rate between the old and new pages.

> Talking about the differences between the $p_values$, in Part II the null hypothesis was that the old page had a better or equal conversion than the new page, and alternative hypotesis was that the new page has a better conversion rate. This means that the hypotesis was a right-sided tail, as we are interested in the right-tail of the distribution. In Part III, as the chosen method was the *Logistic Regression*, the *p-test* is made as a two-sided tail, as in our null hypotesis we consider both old and new pages to have equal conversion rate, and on alternative hypotesis that they have different metrics.

**f.** Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

> Is almost always better to consider other factors about the behavior of an individual (in this case the conversion rate) because in this way we can try to relate multiple variables to our expected result. In this dataset we could also use the timestamp, trying to figure out if an individual is more probable to convert during the day or the night in each of the pages, what could help us understand better and deeper our results.

> But, adding more factors to our model naturally will increase the difficulty in interpreting results, as to understand one variable we need to make constant all the others. Also, we can have what is called the *multicollinearity*, that happens when "independent" variables are related to each other, what can lead us to misleading results.

**g. Adding countries**<br> 
Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. 

1. You will need to read in the **countries.csv** dataset and merge together your `df2` datasets on the appropriate rows. You call the resulting dataframe `df_merged`. [Here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) are the docs for joining tables. 

2. Does it appear that country had an impact on conversion?  To answer this question, consider the three unique values, `['UK', 'US', 'CA']`, in the `country` column. Create dummy variables for these country columns.  

 Provide the statistical output as well as a written response to answer this question.

In [38]:
# Read the countries.csv
df_countries = pd.read_csv('countries.csv')
df_countries.head()

user_id country
0   834778      UK
1   928468      US
2   822059      UK
3   711597      UK
4   710616      UK

In [39]:
df2.head()

user_id                   timestamp      group landing_page  converted  \
0   851104  2017-01-21 22:11:48.556739    control     old_page          0   
1   804228  2017-01-12 08:01:45.159739    control     old_page          0   
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0   
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0   
4   864975  2017-01-21 01:52:26.210827    control     old_page          1   

   intercept  ab_page  
0          1        0  
1          1        0  
2          1        1  
3          1        1  
4          1        0

In [40]:
# Join with the df2 dataframe
df_merged = df2.merge(df_countries, on='user_id')
df_merged.head()

user_id                   timestamp      group landing_page  converted  \
0   851104  2017-01-21 22:11:48.556739    control     old_page          0   
1   804228  2017-01-12 08:01:45.159739    control     old_page          0   
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0   
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0   
4   864975  2017-01-21 01:52:26.210827    control     old_page          1   

   intercept  ab_page country  
0          1        0      US  
1          1        0      US  
2          1        1      US  
3          1        1      US  
4          1        0      US

In [41]:
# Create the necessary dummy variables
df_merged[['CA', 'UK', 'US']] = pd.get_dummies(df_merged['country'])

# Dropping the third column to make sure that
# the model will have a reliable solution (full rank matrix)
df_merged = df_merged.drop('US', axis=1)

df_merged.head()

user_id                   timestamp      group landing_page  converted  \
0   851104  2017-01-21 22:11:48.556739    control     old_page          0   
1   804228  2017-01-12 08:01:45.159739    control     old_page          0   
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0   
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0   
4   864975  2017-01-21 01:52:26.210827    control     old_page          1   

   intercept  ab_page country  CA  UK  
0          1        0      US   0   0  
1          1        0      US   0   0  
2          1        1      US   0   0  
3          1        1      US   0   0  
4          1        0      US   0   0

In [42]:
lm = sm.Logit(df_merged['converted'], df_merged[['intercept', 'CA', 'UK']])
results = lm.fit()
results.summary2()

Optimization terminated successfully.
         Current function value: 0.366116
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            Pseudo R-squared: 0.000      
Dependent Variable: converted        AIC:              212780.8333
Date:               2021-11-24 22:33 BIC:              212812.5723
No. Observations:   290584           Log-Likelihood:   -1.0639e+05
Df Model:           2                LL-Null:          -1.0639e+05
Df Residuals:       290581           LLR p-value:      0.19835    
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     6.0000                                        
-------------------------------------------------------------------
              Coef.   Std.Err.      z      P>|z|    [0.025   0.975]
-------------------------------------------------------------------
intercept    -1.9967    0.0068  -292.3145  0.0000  -2.0101  -1.9833
CA           -0.0408    0.0269    -1.5178  0.1291  -0.0935   0.0119
UK            0.0099    0.0133     0.7458  0.4558  -0.0161   0.0360
==================================================================

"""

In [43]:
np.exp(-0.0408), np.exp(0.0099) # getting the real coefficients

(0.9600211149716509, 1.0099491671175422)

In [44]:
1/np.exp(-0.0408) # as the multiplicative coef. was less than 1, then got the reciprocal

1.0416437559600236

> An user from *US* has 1.04% higher chance to convert than an user from *CA*

**h. Fit your model and obtain the results**<br> 
Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if are there significant effects on conversion.  **Create the necessary additional columns, and fit the new model.** 


Provide the summary results (statistical output), and your conclusions (written response) based on the results. 

In [45]:
df_merged['CA_ab_page'] = df_merged['CA'] * df_merged['ab_page']
df_merged['UK_ab_page'] = df_merged['UK'] * df_merged['ab_page']
df_merged.head()

user_id                   timestamp      group landing_page  converted  \
0   851104  2017-01-21 22:11:48.556739    control     old_page          0   
1   804228  2017-01-12 08:01:45.159739    control     old_page          0   
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0   
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0   
4   864975  2017-01-21 01:52:26.210827    control     old_page          1   

   intercept  ab_page country  CA  UK  CA_ab_page  UK_ab_page  
0          1        0      US   0   0           0           0  
1          1        0      US   0   0           0           0  
2          1        1      US   0   0           0           0  
3          1        1      US   0   0           0           0  
4          1        0      US   0   0           0           0

In [46]:
# Fit your model, and summarize the results
lm = sm.Logit(df_merged['converted'], df_merged[['intercept', 'ab_page', 'CA', 'UK', 'CA_ab_page', 'UK_ab_page']])
results = lm.fit()
results.summary2()

Optimization terminated successfully.
         Current function value: 0.366109
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
==================================================================
Model:              Logit            Pseudo R-squared: 0.000      
Dependent Variable: converted        AIC:              212782.6602
Date:               2021-11-24 22:33 BIC:              212846.1381
No. Observations:   290584           Log-Likelihood:   -1.0639e+05
Df Model:           5                LL-Null:          -1.0639e+05
Df Residuals:       290578           LLR p-value:      0.19199    
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     6.0000                                        
-------------------------------------------------------------------
              Coef.   Std.Err.      z      P>|z|    [0.025   0.975]
-------------------------------------------------------------------
intercept    -1.9865    0.0096  -206.3440  0.0000  -2.0053  -1.9676
ab_page      -0.0206    0.0137    -1.5052  0.1323  -0.0473   0.0062
CA           -0.0175    0.0377    -0.4652  0.6418  -0.0914   0.0563
UK           -0.0057    0.0188    -0.3057  0.7598  -0.0426   0.0311
CA_ab_page   -0.0469    0.0538    -0.8718  0.3833  -0.1523   0.0585
UK_ab_page    0.0314    0.0266     1.1807  0.2377  -0.0207   0.0835
==================================================================

"""

In [47]:
def print_exp_coef(text, coef):
    s = text + ' coef:'
    exp_coef = np.exp(coef)
    
    s += f' {round(exp_coef, 5)}'
    
    if exp_coef < 1:
        rec_exp_coef = 1 / exp_coef
        s += f' - reciprocal: {round(rec_exp_coef, 5)}'
    
    print(s)

In [48]:
print_exp_coef('ab_page', -0.0206)
print_exp_coef('CA', -0.0175)
print_exp_coef('UK', 0.0057)
print_exp_coef('CA_ab_page', -0.0469)
print_exp_coef('UK_ab_page', 0.0314)

ab_page coef: 0.97961 - reciprocal: 1.02081
CA coef: 0.98265 - reciprocal: 1.01765
UK coef: 1.00572
CA_ab_page coef: 0.95418 - reciprocal: 1.04802
UK_ab_page coef: 1.0319


In [49]:
print(f'First record: {df_merged.timestamp.min()[:10]}')
print(f'Last record: {df_merged.timestamp.max()[:10]}')

First record: 2017-01-02
Last record: 2017-01-24


> Looking at all the $p_{values}$ in the regression model, all of them are higher than the Error Type I threshold ($\alpha = 0.05$). This means that we fail to reject $H_0$, that in this case means that we do not have enough statistical evidence to say that the conversion rate between the new page and the old page are different.

> At this time, I made a High-Order term regarding the interaction between the column `ab_page` and `UK`/`CA`, to see if together they are useful for the analysis, and they are gotten by multiplying the `ab_page` with the `country` column.

> Looking at the coefficients, we find the following differences:

* Users with the **old page** have 1.02% higher chance to convert than users with the **new page**.
* Users from **US** have 1.02% higher chance to convert than users from **CA**.
* Users from **UK** have 1.01% higher chance to convert than users from **US**.
* Users from **US** with **new page** have 1.05% higher of chance to convert than users from **CA** with **new page**.
* Users from **UK** with **new page** have 1.03% higher chance to convert than users from **US** with **new page**.

> Although it represents a statistical difference of almost 1% in all cases, in practice this usually is not enough to justify the resources spent in such a development. Developing and studying a new landing page is something that takes time, and a 1% difference in the convertion rate (especially in small/medium companies) is not high enough to pay for the changes.

> We can see that the data was collected during 22 days, what means that maybe a longer A/B Testing could helps us understand better the conversion. Looking that by our model, the **old page** has around 1% better performance, and this may be explained that users that were used with the old page has some kind of resistence with the new layout. Extending the testing period, we could reduce this factor, and maybe see different results from our model.